# for Greenwood and Jovanovich


Other refereces:
- Integral over a distribution: http://math.stackexchange.com/questions/380785/what-does-it-mean-to-integrate-with-respect-to-the-distribution-function


In [ ]:
import numpy as np
import scipy as sp
import scipy.stats
from scipy.optimize import fminbound

In [ ]:
# Interpolation

grid = np.linspace(0,10,11)
w = 5 * grid - 25
print sp.interp(2, grid, w)
print type(grid)
print grid.size


In [ ]:
# Normal distributions density funciton.
import scipy.stats

def exp_val(x,safe):
    print x,":",max(safe,x),"*",sp.stats.norm(loc=2,scale=0.3).pdf(x),"=",((max(safe,x))*sp.stats.norm(loc=2,scale=0.3).pdf(x))
    return ((max(safe,x))*sp.stats.norm(loc=2,scale=0.3).pdf(x))
    
def int_exp_val(safe):
    return sp.integrate.quad(exp_val,0,5,args=(safe))[0]
    
grid = np.random.normal(2,0.3,size=60)

wealth_distr = scipy.stats.norm(loc=2,scale=0.4)

#print min(grid)
print int_exp_val(0)
print sp.integrate.quad(lambda x: sp.stats.norm(loc=2,scale=0.3).pdf(x),0,5)[0]


In [ ]:
print sp.stats.norm(loc=2,scale=0.3).pdf(4.191)

Tests with the interpolation function and the integral function.

In [ ]:
    
x_axis = np.linspace(0,15,500)
y_axis = x_axis*0.5
print y_axis

In [ ]:
def x_interp(xax,yax):
    return lambda x: sp.interp(x,xax,yax)

function_ = x_interp(x_axis,y_axis)
print function_(15)


In [ ]:
SAFE = 1
def PDF(x):
    return sp.stats.norm(loc=2,scale=0.3).pdf(x)
def integral(s,am):
    return sp.integrate.quad((lambda theta,sI : sp.interp(sI*max(SAFE,theta),x_axis,y_axis)*PDF(theta)),0,am,args=s)

print integral(1,10)

In [ ]:
BETA = 0.987
capital = 3
objective = lambda s,k: - np.log(k-s) - BETA * integral(s,10)[0]
s_star = fminbound(objective, 1e-6, capital, args=(capital,))
print s_star
print -objective(s_star,capital)

## The Bellman operators for the two value functions
This part is based on the Prof. Sargent and Prof. Stachurski's approach from [HERE](http://quant-econ.net/py/optgrowth.html).

### The Bellman operator for V(K)
THIS IS A FINAL VERSION FOR THE the V(K) FUNCTION!

In [ ]:
import numpy as np
import scipy as sp
import scipy.stats
import parameters as param
from scipy.integrate import quad
from scipy.optimize import fminbound
from numpy import log
from scipy import interp


SAFE = param.SAFE_R
BETA = param.BETA
# DISTRIBUTION PARAM OF THETA (Aggregate shock)
MEAN = param.AG_MEAN
STDEV = param.AG_STDE
MAX_VAL = param.AG_MAXVAL   # Max value of theta for the integration
# Number of iterations
N = 2

wealth_axis = np.linspace(1e-6,param.K_MAX,2*param.K_MAX)

def PDF(x):
    return scipy.stats.norm(loc=MEAN,scale=STDEV).pdf(x)

In [ ]:

def v_integral(s,W_ax,Y_ax):
    return quad((lambda theta,sI : interp(sI*max(SAFE,theta),
                                        W_ax,Y_ax)*PDF(theta)),0,MAX_VAL,args=(s,),limit=100)

def v_bellman_op(v):
    Tv = np.empty(wealth_axis.size)
    policy_func = np.empty(wealth_axis.size)
    for i, k in enumerate(wealth_axis):
        objective = lambda s,k: - np.log(k-s) - BETA * v_integral(s,wealth_axis,v)[0]
        s_star = fminbound(objective, 1e-12, k-1e-12, args=(k,))
        Tv[i] = -objective(s_star,k)
    return Tv

def policy(v):
    policy_f = np.empty(wealth_axis.size)
    for i,k in enumerate(wealth_axis):
        objective = lambda s,k: -np.log(k-s)-BETA*v_integral(s,)
        policy_f[i] = fminbound(objective, 1e-12,k-1e-12, args=(k,))
    return policy_f
    
v =  76.9230769 * log(wealth_axis) + 3569.764136
time_path = [[] for i in range(N)]
for i in range(N):
    print ">>>> Iteration No. ",i
    v = v_bellman_op(v)
    time_path[i] = v
greedy__policy = policy(v)

np.savetxt('wealth_grid.csv',wealth_axis, delimiter=",")
print "Return wealth grid in 'wealth_grid.csv'."
np.savetxt('v_value_func.csv',v, delimiter=",")
print "Return value function v in 'v_value_func.csv'."
np.savetxt('v_policy_func.csv',greedy_policy,delimiter=',')
print "Return v greedy policy function in 'v_policy_func.csv'."
np.savetxt('v_time_path.csv',time_path,delimiter=",")


The multiprocessing version can be found in the python file.
In order to improve the iteration, the coefficients of the value function using the analytical solution is:

In [ ]:
integr = quad(lambda theta: log(max(SAFE,theta))*PDF(theta),0,MAX_VAL,limit=100)[0]
constant = (1/(1-BETA))*log(1-BETA) + (BETA / ((1-BETA)**2)) * log(BETA) + \
           (BETA / ((1-BETA)**2))  * integr

coeff1 = (1 / (1-BETA))
print integr
print constant
print coeff1

### The Bellman operator for W(K)

In [6]:
import numpy as np
import scipy as sp
import scipy.stats
import parameters as param
from scipy.integrate import dblquad
from scipy.optimize import fminbound
from numpy import log
from scipy import interp

SAFE = param.SAFE_R
BETA = param.BETA
# DISTRIBUTION PARAM (AGGREGATE)
AG_MEAN = param.AG_MEAN
AG_STDEV = param.AG_STDE
MAX_VAL_AG = param.AG_MAXVAL
# DISTRIBUTION PARAM (idios. shocks)
ID_MEAN = param.IS_MEAN
ID_STDEV = param.IS_STDE
MAX_VAL_E = param.IS_MAXVAL
MIN_VAL_E = param.IS_MINVAL
# Number of iterations
N = 1
COST = param.COST_INT

wealth_axis = np.linspace(1e-6,param.K_MAX,3) # np.ceil(param.K_MAX*0.75)
v_func = 76.9230769* log(wealth_axis) + 3569.764136

def aggPDF(x):
    return scipy.stats.norm(loc=AG_MEAN,scale=AG_STDEV).pdf(x)
def idiPDF(x):
    return scipy.stats.norm(loc=ID_MEAN,scale=ID_STDEV).pdf(x)

def w_integral(s,W_ax,Y_ax):
    # *aggPDF(theta)*idiPDF(eps)
    function = lambda theta,eps,s: max(interp(s*(theta+eps),W_ax,Y_ax),interp(s*(theta+eps)-COST,W_ax,v_func))
    return dblquad(function,MIN_VAL_E,MAX_VAL_E,lambda x: 0, lambda x: MAX_VAL_AG, args=(s,))

def w_bellman_op(w):
    Tw = np.empty(wealth_axis.size)
    for i,k in enumerate(wealth_axis):
        print "Point ",i+1," of ",len(wealth_axis),"."
        objective = lambda s,k: - log(k-s) - BETA * w_integral(s,wealth_axis,w)[0]
        s_star = fminbound(objective, 1e-12, k-1e-12, args=(k,))
        Tw[i] = -objective(s_star,k)
    return Tw

w = 10*log(wealth_axis) + 10
for i in range(N):
    print i
    w = w_bellman_op(w)
np.savetxt('w_value_func.csv',w,delimiter=',')

0
Point  1  of  3 .
Point  2  of  3 .


C:\Anaconda2\lib\site-packages\scipy\integrate\quadpack.py:357: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  warnings.warn(msg, IntegrationWarning)


KeyboardInterrupt: 